<a href="https://colab.research.google.com/github/hwarang97/Pytorch_introduction/blob/main/pytorch_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 신경망 구축

- torch.nn  --> 신경망 구성 요소 제공
- nn.Module --> 모든 신경망 계층 다룸

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader # 데이터셋을 이터러블 객체로 만드는데 사용
from torchvision import datasets, transforms # datasets : 데이터셋 제공

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device') # gpu 사용 가능

Using cuda device


## 클래스 정의하기

1. 신경망 모델을 nn.Module의 하위 클래스로 선언
2. __init__ 에서 신경망 계층들을 초기화
3. nn.Module 을 상속받은 모든 클래스는 forward 메소드로 입력 데이터로 연산 수행

In [3]:
class NN(nn.Module):
  def __init__(self):
    super(NN, self).__init__() # super 매개변수로 (하위클래스 이름, 하위클래스 객체) 가 넘겨진다 
                               # 보통 부모 - 자식관계이므로, 생략하면 자식이 부모를 호출한다고 인식하고 생략되기도 함
                               # 만약 조부모(A) - 부모(B) - 자식(C) 관계라고 한다면
                               # 자식 선언시 매개변수를 생략하면 부모클래스를 호출하는것이 됨
                               # 그러나 명시적으로 자식클래스에서 조부모(A)를 매개변수로 넣으면 super(B, self)
                               # 부모(B)의 부모 클래스인 조부모(A) 클래스를 호출하는 것이 됨
                               # 즉, 상황에 따른 설정이 가능하다는 것!!
                               # 결론 : super(하위 클래스, 하위 클래스 객체인데 직속 하위가 아니여도 됨) 

    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512), # 28*28 개의 픽셀값을 입력받음 --> 512개의 뉴런에게 전달
        nn.ReLU(),            
        nn.Linear(512,512),    # 512개 뉴런 --> 512개 뉴런
        nn.ReLU(),
        nn.Linear(512,10),     # 512개 뉴런 --> 10개 뉴런 (분류할 이미지가 10종류라서)
    )

  def forward(self, x):                # 실제 모델 연산 구현
    x = self.flatten(x)                # flatten 실행
    logits = self.linear_relu_stack(x) # 설정한 linear, relu 연산 실행
    return logits                      # 각 클래스별로 logit 값이 나옴 --> softmax 함수에 넣어서 확률로 변환

In [4]:
model = NN().to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# 임의의 이미지를 만들어서 작동시켜보기
X = torch.rand(1, 28, 28, device=device)
logits = model(X)                       # 각 클래스별 로짓값, tensor([[0.0015, 0.0578, ..., -0.0096]])
pred_probab = nn.Softmax(dim=1)(logits) # 각 클래스별 확률 반환, tensor([[0.0992, 0.1050, ... , 0.0981]]) -> 2차원 배열 형식
y_pred = pred_probab.argmax(1)          # 가장 큰 원소를 반환. numpy 계열 함수 <--> argmin 
                                        # dim=1은 열끼리 비교하도록 설정
print(f'Predicted class : {y_pred}')

Predicted class : tensor([0], device='cuda:0')


### Softmax(dim=1)(logits) 조사
- 형식이 생소했고, 왜 이렇게 만들어졌는지 모르겠었음
  - 내가 알던 양식은 객체 선언 시, 매개변수에 다 같이 넣는것 (softmax(logits, dim=1)    

---
- 조사한 것
- SoftMax.__init__(self, dim) : 객체 선언시 받을 수 있는 인자는 최대 2개뿐. (logits는 생성시 같이 사용할 수 없음)
- SoftMax.forward(self, input : Tensor) -> Tensor : 인자를 받아서 softmax 연산을 해주는 것을 보임. 아마도 logits을 받는 함수인듯
- 결론 : nn.Softmax(dim=1) 은 객체를 생성한 것이고, 이후 객체 함수 forward를 이용해서 logits 을 처리한것

## 모델 계층(Layer)
- 신경망에서 어떤 일이 벌어지는지 확인해보기

In [6]:
# 임의의 입력 이미지 생성
input_image = torch.rand(3, 28, 28) # 28*28 이미지 3장
print(input_image.size())

torch.Size([3, 28, 28])


### Flatten
- Flatten 계층
- 2D 이미지 --> 배열

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### Linear
- 선형 계층
- 뉴런별로 입력값에 가중치와 편향 값을 사용하여 선형방정식 만들어주는 모듈(코드)

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size()) # tensor([3,20]), 선형 결과값이 이미지당 1개씩 총 3개

torch.Size([3, 20])


### ReLU
- 활성와 함수, 비선형성
- 더 깊이있게 학습시키 위해서는 비선형성이 필요해서 도입됨

In [9]:
print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)

Before ReLU : tensor([[-6.6460e-01,  1.4998e-01,  3.1055e-01,  5.2512e-02, -5.3694e-01,
         -1.7473e-01, -3.7515e-01, -5.0756e-02, -2.0374e-01,  2.3192e-01,
         -1.9434e-01,  7.2228e-01,  3.8014e-01, -2.5898e-01, -2.7894e-01,
          5.0681e-01,  6.8300e-02, -3.2887e-01, -1.4381e-01,  8.3760e-03],
        [-2.5751e-01,  4.7884e-01,  5.0942e-01,  5.6973e-02, -3.1149e-01,
         -2.8223e-01, -4.6009e-01, -1.7541e-01,  5.9050e-02, -7.0161e-02,
         -1.4262e-01,  4.2949e-01,  3.2752e-01, -1.5422e-01,  4.0095e-04,
         -1.0780e-01,  2.4495e-02, -5.1550e-01, -4.4753e-01,  4.8999e-02],
        [-1.1676e-02,  2.8649e-01,  7.7862e-01,  4.4517e-01, -1.1189e-01,
         -5.3759e-01, -6.9004e-01, -1.5298e-01, -1.7292e-01,  1.1745e-01,
          2.0234e-01,  4.3492e-01, -1.4537e-01, -5.8344e-01, -3.4567e-01,
          4.8129e-01,  6.7047e-01, -6.6153e-01, -8.7367e-02, -1.5606e-01]],
       grad_fn=<AddmmBackward0>)




##  Sequential
- 순서가 있는 모듈 컨테이너(= 사용하기 편하도록 미리 작성한 코드들을 순서대로 담아주는 객체)
- 이걸 통해서 신경망 만들 수 있음

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## Softmax
- logit 값을 반환해줌(= 선형값, 뉴런에 들어온값,을 로지스틱 함수,S자 모양, 에 넣어서 각 클래스별로 확률 값으로 바꿔줌. 이걸 osftmax가 합이 1이 되도록 클래스별로 확률을 조정해줌

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 모델 매개변수
- 신경망의 계층들의 모든 것들이 저장되며, 매개변수를 통해서 확인할 수 있음

In [14]:
print(f'Model structure: {model}\n\n')

for name, param in model.named_parameters():
  print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')

Model structure: NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0190,  0.0250, -0.0021,  ..., -0.0141, -0.0009, -0.0172],
        [ 0.0205, -0.0102, -0.0109,  ..., -0.0199, -0.0203, -0.0348]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0139, -0.0155], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0030,  0.0298, -0.0315,  ...,  0.0227,  0.0095,  0.0005],
        [-0.0214, -0.0368, -0.0190,  ...,  0.0040, -0.0295,  0.0089]],
       device='cuda:0', grad_fn=<SliceBackward0>)